In [1]:
import numpy as np
import json

In [2]:
with open("opinion_chunks.json", "r") as f:
    chunks = json.load(f)

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [4]:
chunks[0]["text"]

'1 Stephanie Sheridan (CA 135910) Meegan B. Brooks (CA 298570) 2 Benesch, Friedlander, Coplan & Aronoff LLP 100 Pine Street, Suite 3100 3 San Francisco, California 94111 Telephone: 628.600.2250 4 Facsimile: 628.221.5828 Email: ssheridan@beneschlaw.com 5 mbrooks@beneschlaw.com Attorneys for Defendant 7 LOWE’S HOME CENTERS, LLC UNITED STATES DISTRICT COURT EASTERN DISTRICT OF CALIFORNIA SACRAMENTO DIVISION PRESCILA LOVELL, for herself, as a Case No. 2:25-cv-03453-TLN-SCR 13 private attorney general, and on behalf of all others similarly situated, 14 STIPULATION AND ORDER TO EXTEND Plaintiff, TIME FOR DEFENDANT LOWE’S HOME 15 CENTERS, LLC TO RESPOND TO v. PLAINTIFF’S COMPLAINT LOWE’S HOME CENTERS, LLC, Defendant. 1 Pursuant to Federal Rule of Civil Procedure 16(b)(4) and Local Rule 144(a), Plaintiff Prescila 2 Lovell (“Plaintiff”) and Defendant Lowe’s Home Centers, LLC (“Defendant”) (collectively, the 3 “Parties”), by and through their respective counsel, hereby stipulate that Defendant’s

In [5]:
for chunk in chunks:
  chunk["embedding"] = model.encode(f"search_document: {chunk["text"]}")

In [6]:
embeddings = None
n = 0
for chunk in chunks:
    array = np.array(chunk["embedding"], dtype=np.float32).reshape(1, -1)
    if embeddings is None:
        embeddings = array/np.linalg.norm(array)
    else:
        embeddings = np.vstack([embeddings, array/np.linalg.norm(array)])
    n += 1

In [7]:
embeddings.shape[1]

768

In [8]:
import faiss

d = embeddings.shape[1]
base = faiss.IndexFlatL2(d)
index = faiss.IndexIDMap(base)
original_indices = np.arange(len(embeddings), dtype=np.int64)

In [9]:
index.add_with_ids(embeddings, original_indices)

In [10]:
faiss.write_index(index, "opinions.index")

In [ ]:
for chunk in chunks:
  chunk["embedding"] = chunk["embedding"].tolist()

with open("opinion_embeddings.json", "w") as f:
  json.dump(chunks, f)

In [11]:
index.d

768